# Homework: Basic Artificial Neural Networks

The goal of this homework is simple, yet an actual implementation may take some time :). We are going to write an Artificial Neural Network (almost) from scratch. The software design was heavily inspired by [PyTorch](http://pytorch.org) which is the main framework of our course 

This homework requires sending **multiple** files, please do not forget to include all the files when sending to TA. The list of files:
- This notebook
- homework_modules.ipynb with all blocks implemented (except maybe `Conv2d` and `MaxPool2d` layers implementation which are part of 'advanced' version of this homework)
- homework_differentiation.ipynb

In [ ]:
%matplotlib inline
from time import time, sleep
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
from collections import defaultdict
from math import prod
from random import random

# Framework

Implement everything in `Modules.ipynb`. Read all the comments thoughtfully to ease the pain. Please try not to change the prototypes.

Do not forget, that each module should return **AND** store `output` and `gradInput`.

The typical assumption is that `module.backward` is always executed after `module.forward`,
so `output` is stored, this would be useful for `SoftMax`. 

### Tech note
Prefer using `np.multiply`, `np.add`, `np.divide`, `np.subtract` instead of `*`,`+`,`/`,`-` for better memory handling.

Example: suppose you allocated a variable 

```
a = np.zeros(...)
```
So, instead of
```
a = b + c  # will be reallocated, GC needed to free
``` 
You can use: 
```
np.add(b,c,out = a) # puts result in `a`
```

In [ ]:
# (re-)load layers
%run homework_modules.ipynb

# Toy example

Use this example to debug your code, start with logistic regression and then test other layers. You do not need to change anything here. This code is provided for you to test the layers. Also it is easy to use this code in MNIST task.

In [ ]:
# Generate some data
N = 500

X1 = np.random.randn(N,2) + np.array([2,2])
X2 = np.random.randn(N,2) + np.array([-2,-2])

Y = np.concatenate([np.ones(N),np.zeros(N)])[:,None]
Y = np.hstack([Y, 1-Y])

X = np.vstack([X1,X2])
plt.scatter(X[:,0],X[:,1], c = Y[:,0], edgecolors= 'none')

Define a **logistic regression** for debugging. 

In [ ]:
net = Sequential()
net.add(Linear(2, 2))
net.add(LogSoftMax())

criterion = ClassNLLCriterion()

# Test something like that then 

# net = Sequential()
# net.add(Linear(2, 4))
# net.add(ReLU())
# net.add(Linear(4, 2))
# net.add(LogSoftMax())

print(net)

Start with batch_size = 1000 to make sure every step lowers the loss, then try stochastic version.

In [ ]:
# Optimizer params
optimizer_config = {'learning_rate' : 1e-1, 'momentum': 0.9}
optimizer_state = {}

# Looping params
n_epoch = 5
batch_size = 10

In [ ]:
# batch generator
def get_batches(dataset, batch_size):
    X, Y = dataset
    n_samples = X.shape[0]
        
    # Shuffle at the start of epoch
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        
        batch_idx = indices[start:end]
    
        yield X[batch_idx], Y[batch_idx]

### Train

Basic training loop. Examine it.

In [ ]:
def training_loop(X, Y, net, opt, opt_config, criterion, n_epoch, batch_size):
    opt_state = {}
    
    for i in range(n_epoch):
        for x_batch, y_batch in get_batches((X, Y), batch_size):

            net.zeroGradParameters()

            # Forward
            predictions = net.forward(x_batch)
            yield criterion.forward(predictions, y_batch)

            # Backward
            dp = criterion.backward(predictions, y_batch)
            net.backward(x_batch, dp)

            # Update weights
            opt(net.getParameters(), net.getGradParameters(), opt_config, opt_state)


In [ ]:
loss_history = []

for loss in training_loop(X, Y, net=net, opt=sgd_momentum, opt_config=optimizer_config,
                          criterion=criterion, n_epoch=n_epoch, batch_size=batch_size):
    loss_history.append(loss)

    # Visualize
    display.clear_output(wait=True)
    plt.figure(figsize=(8, 6))
        
    plt.title("Training loss")
    plt.xlabel("#iteration")
    plt.ylabel("loss")
    plt.plot(loss_history, 'b')
    plt.show()
    
    print('Current loss: %f' % loss)    

# Digit classification 

We are using old good [MNIST](http://yann.lecun.com/exdb/mnist/) as our dataset.

In [ ]:
import mnist
X_train, y_train, X_val, y_val, X_test, y_test = mnist.load_dataset()

One-hot encode the labels first.

In [ ]:
# Your code goes here. ################################################
def one_hot_encode(y, n_classes):
    Y = np.zeros(y.shape + (n_classes,))
    Y[range(Y.shape[0]), y] = 1
    return Y

Y_train = one_hot_encode(y_train, 10)
Y_val = one_hot_encode(y_val, 10)
Y_test = one_hot_encode(y_test, 10)

def flatten(X):
    return X.reshape((X.shape[0], prod(X.shape[1:])))

X_train_f = flatten(X_train)
X_val_f = flatten(X_val)
X_test_f = flatten(X_test)

- **Compare** `ReLU`, `ELU`, `LeakyReLU`, `SoftPlus` activation functions. 
You would better pick the best optimizer params for each of them, but it is overkill for now. Use an architecture of your choice for the comparison.
- **Try** inserting `BatchNormalization` (folowed by `ChannelwiseScaling`) between `Linear` module and activation functions.
- Plot the losses both from activation functions comparison and `BatchNormalization` comparison on one plot. Please find a scale (log?) when the lines are distinguishable, do not forget about naming the axes, the plot should be goodlooking.
- Plot the losses for two networks: one trained by momentum_sgd, another one trained by Adam. Which one performs better?
- Hint: good logloss for MNIST should be around 0.5. 

In [ ]:
def mk_net(act_layer, batch_norm=False):
    n_in = X_train_f.shape[1]
    n_hidden = 256
    
    net = Sequential()
    net.add(Linear(n_in, n_hidden))
    if batch_norm:
        net.add(BatchNormalization(0.9))
        net.add(ChannelwiseScaling(n_hidden))
    net.add(act_layer)
    net.add(Linear(n_hidden, 10))
    net.add(LogSoftMax())
    
    return net

config = {
    'ReLU': {
        'net': mk_net(ReLU()),
        'color': 'b'
    },
    'ELU': {
        'net': mk_net(ELU()),
        'color': 'r'
    },
    'LeakyReLU': {
        'net': mk_net(LeakyReLU()),
        'color': 'g'
    },
    'SoftPlus': {
        'net': mk_net(SoftPlus()),
        'color': 'y'
    },
}

def visualise_history(loss_history):
    # Visualize
    display.clear_output(wait=True)
    f = plt.figure(figsize=(8, 6))

    plt.title("Training loss")
    plt.xlabel("#iteration")
    plt.ylabel("log-loss")
    plt.yscale('log')
    for name in config:
        plt.plot(loss_history[name], config[name]['color'])
    plt.show()

    for name in config:
        print('Current %s loss: %f' % (name, loss_history[name][-1]))


def visualise_loss(opt, opt_config):
    loss_history = defaultdict(lambda: [])
    loops = {name:training_loop(X_train_f, Y_train, net=conf['net'],
                                opt=opt, opt_config=opt_config,
                                criterion=criterion, n_epoch=n_epoch, batch_size=batch_size)
             for name, conf in config.items()}
    try:
        while True:
            for name, loss in loops.items():
                loss_history[name].append(next(loss))

            if random() > max(batch_size / y_train.shape[0] * 100, 0.01):
                continue
            
            visualise_history(loss_history)

    except StopIteration:
        pass

    visualise_history(loss_history)


visualise_loss(sgd_momentum, optimizer_config)

In [ ]:
visualise_loss(adam_optimizer, {'learning_rate': 1e-3, 'beta1': 0.9, 'beta2':0.999, 'epsilon':1e-8})

Write your personal opinion on the activation functions, think about computation times too. Does `BatchNormalization` help?

In [ ]:
# Your answer goes here. ################################################

**Finally**, use all your knowledge to build a super cool model on this dataset. Use **dropout** to prevent overfitting, play with **learning rate decay**. You can use **data augmentation** such as rotations, translations to boost your score. Use your knowledge and imagination to train a model. Don't forget to call `training()` and `evaluate()` methods to set desired behaviour of `BatchNormalization` and `Dropout` layers.

In [ ]:
# Your code goes here. ################################################

Print here your accuracy on test set. It should be around 90%.

In [ ]:
# Your answer goes here. ################################################